En el presente código se entrenan y evalúan tres modelos de clasificación de XGBoost con 10 árboles, donde como variables independientes se utilizan los valores de diferentes variables meteorológicas para cada uno de los diferentes días anteriores al momento de la predicción. La variable objetivo es el nivel cualitativo de la precipitación del día siguiente en términos de los percentiles de la Climatología, clasificada de la siguiente forma:

Clase -1: Si la lluvia del día siguiente es inferior o igual al P(1/3)

Clase 0: Si la lluvia del día siguiente es mayor al P(1/3) y menor al P(2/3)

Clase +1: Si la lluvia del día siguiente es superior o igual al P(2/3)

Un modelo es entrenado con datos de todo el año, el segundo con datos solo de la época lluviosa, y el tercero con datos de la época seca. 

Se realizan tres validaciones para cada modelo: una con datos de todo el año, una con datos exclusivos de la época lluviosa y la última con datos de la época seca.

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# IMPORTACIÓN Y PREPARACIÓN DE DATOS

In [ ]:
import pandas as pd
import numpy as np

# Se importan los datos de entrenamiento de todo el año
file_path = 'Train_all.xlsx'
trainall = pd.read_excel(file_path)

# Se importan los datos de validación de todo el año
file_path = 'Test_all.xlsx'
testall = pd.read_excel(file_path)

#--------------------------------------------------------------------

# Se importan los datos de entrenamiento de la época lluviosa
file_path = 'Train_rainy.xlsx'
trainrainy = pd.read_excel(file_path)

# Se importan los datos de validación de la época lluviosa
file_path = 'Test_rainy.xlsx'
testrainy = pd.read_excel(file_path)

#--------------------------------------------------------------------

# Se importan los datos de entrenamiento de la época seca
file_path = 'Train_dry.xlsx'
traindry = pd.read_excel(file_path)

# Se importan los datos de validación de la época seca
file_path = 'Test_dry.xlsx'
testdry = pd.read_excel(file_path)

### SEPARACIÓN DE DATOS EN X,Y

In [ ]:
# Separamos los datos de entrenamiento de todo el año en variables independientes (X_train) y variable objetivo (Y_train)
Y_trainall = pd.DataFrame()
Y_trainall['percentil'] = trainall['percentil']
X_trainall = trainall.drop(columns='percentil')

# Separamos los datos de validación de todo el año en variables independientes (X_test) y variable objetivo (Y_test)
Y_testall = pd.DataFrame()
Y_testall['percentil'] = testall['percentil']
X_testall = testall.drop(columns='percentil')

#--------------------------------------------------------------------------------------------------------------------------

# Separamos los datos de entrenamiento de época lluviosa en variables independientes (X_train) y variable objetivo (Y_train)
Y_trainrainy = pd.DataFrame()
Y_trainrainy['percentil'] = trainrainy['percentil']
X_trainrainy = trainrainy.drop(columns='percentil')

# Separamos los datos de validación de la época lluviosa en variables independientes (X_test) y variable objetivo (Y_test)
Y_testrainy = pd.DataFrame()
Y_testrainy['percentil'] = testrainy['percentil']
X_testrainy = testrainy.drop(columns='percentil')

#--------------------------------------------------------------------------------------------------------------------------

# Separamos los datos de entrenamiento de época seca en variables independientes (X_train) y variable objetivo (Y_train)
Y_traindry = pd.DataFrame()
Y_traindry['percentil'] = traindry['percentil']
X_traindry = traindry.drop(columns='percentil')

# Separamos los datos de validación de la época seca en variables independientes (X_test) y variable objetivo (Y_test)
Y_testdry = pd.DataFrame()
Y_testdry['percentil'] = testdry['percentil']
X_testdry = testdry.drop(columns='percentil')

#--------------------------------------------------------------------------------------------------------------------------

### MAPEAR LAS CLASES

In [ ]:
# XGB Classifier espera que las clases sean valores enteros que comiencen desde 0 y se incrementen secuencialmente (0,1,2...)
# Por lo tanto, hacemos el siguiente mapeo de clases:

# -1 -> 0
# 0 -> 1
# +1 -> 2

# Definir el mapeo
class_mapping = {-1: 0, 0: 1, 1: 2}

# Aplicar el mapeo a la variable objetivo

Y_trainall_mapped = Y_trainall['percentil'].map(class_mapping)
Y_testall_mapped = Y_testall['percentil'].map(class_mapping)

Y_trainrainy_mapped = Y_trainrainy['percentil'].map(class_mapping)
Y_testrainy_mapped = Y_testrainy['percentil'].map(class_mapping)

Y_traindry_mapped = Y_traindry['percentil'].map(class_mapping)
Y_testdry_mapped = Y_testdry['percentil'].map(class_mapping)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE TODO EL AÑO

### ENTRENAMIENTO DEL MODELO

In [ ]:
import xgboost as xgb
import joblib
import time

modelo = xgb.XGBClassifier(n_estimators=10)

start_time = time.time()
modelo.fit(X_trainall, Y_trainall_mapped)
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'XGBoost_10trees_comb1_5days_alltrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
# Importancia de características (Feature importance)
nombres_columnas = X_trainall.columns
importancia = modelo.feature_importances_

print('\nImportancia de cada característica:')
for nombre, imp in zip(nombres_columnas, importancia):
    print(f'{nombre}: {imp:.4f}')

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predall_mapped = modelo.predict(X_testall)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testall_mapped, Y_predall_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA-----------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predrainy_mapped = modelo.predict(X_testrainy)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testrainy_mapped, Y_predrainy_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA---------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_preddry_mapped = modelo.predict(X_testdry)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testdry_mapped, Y_preddry_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE LA ÉPOCA LLUVIOSA

### ENTRENAMIENTO DEL MODELO

In [ ]:
import xgboost as xgb
import joblib
import time

modelo = xgb.XGBClassifier(n_estimators=10)

start_time = time.time()
modelo.fit(X_trainrainy, Y_trainrainy_mapped)
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'XGBoost_10trees_comb1_5days_rainytrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
# Importancia de características (Feature importance)
nombres_columnas = X_trainall.columns
importancia = modelo.feature_importances_

print('\nImportancia de cada característica:')
for nombre, imp in zip(nombres_columnas, importancia):
    print(f'{nombre}: {imp:.4f}')

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predall_mapped = modelo.predict(X_testall)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testall_mapped, Y_predall_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA-----------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predrainy_mapped = modelo.predict(X_testrainy)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testrainy_mapped, Y_predrainy_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA---------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_preddry_mapped = modelo.predict(X_testdry)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testdry_mapped, Y_preddry_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# ENTRENAMIENTO CON DATOS DE LA ÉPOCA SECA

### ENTRENAMIENTO DEL MODELO

In [ ]:
import xgboost as xgb
import joblib
import time

modelo = xgb.XGBClassifier(n_estimators=10)

start_time = time.time()
modelo.fit(X_traindry, Y_traindry_mapped)
end_time = time.time()

training_time = end_time - start_time

# Guarda el modelo en un archivo
joblib.dump(modelo, 'XGBoost_10trees_comb1_5days_drytrain.pkl')

print('EL MODELO HA SIDO ENTRENADO EXITOSAMENTE')
print()
print(f'Tiempo de entrenamiento: {training_time:.4f} segundos')

In [ ]:
# Importancia de características (Feature importance)
nombres_columnas = X_trainall.columns
importancia = modelo.feature_importances_

print('\nImportancia de cada característica:')
for nombre, imp in zip(nombres_columnas, importancia):
    print(f'{nombre}: {imp:.4f}')

### EVALUACIÓN DEL MODELO

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE TODO EL AÑO--------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predall_mapped = modelo.predict(X_testall)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testall_mapped, Y_predall_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA LLUVIOSA-----------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_predrainy_mapped = modelo.predict(X_testrainy)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testrainy_mapped, Y_predrainy_mapped)
print("Matriz de Confusión:")
print(conf_matrix)

In [ ]:
#---------------------------------EVALUACIÓN CON DATOS DE LA ÉPOCA SECA---------------------------------------------------------

from sklearn.metrics import confusion_matrix
import numpy as np

# Se realiza la predicción
Y_preddry_mapped = modelo.predict(X_testdry)

# Se calcula y despliega la matriz de confusión
conf_matrix = confusion_matrix(Y_testdry_mapped, Y_preddry_mapped)
print("Matriz de Confusión:")
print(conf_matrix)